In [3]:
"""
Dictates stellar abundances used in calculating ion balances.
If filename = None, solar abundances will be used. None is the default.
Modify_solar = False is the default:
If modify_solar = False, will return data input as a dictionary. Only those elements and values provided will be
    used in calculating ion balances. 
If modify_solar = True, switch out element values provided with solar abundances for those elements and provide
full dictionary of first 30 elements in periodic table having modified abundances provided by data. All else is same
Ratios = True is the default:
If ratios = False, data values will not be changed.
If ratios = True, data values will be adjusted to accomodate chemical abundance ratios (see def fix_elements, 
which is called in def find_elements)
Right now, only works with datasets able to be read in as a pandas dataframe (see read_filetype doc string)
"""
def reading_func(filename = None, modify_solar = False, ratios = True, **kwargs):

    def read_filetype(filename, what_filetype = False, **kwargs):  
    """
    loads data into pandas dataframe
    accaptable filetypes: 
    --> csv, txt, ods, odt, xls, xlsx, xlsm, xlsb, odf, html, fwf, json, orc, h5, hdf5
    """ 
        import pandas as pd
        if what_filetype is not False: #may not even need this but she's here lolololol
            print('accaptable filetypes: csv, txt, ods, odt, xls, xlsx, xlsm, xlsb, odf, html, fwf, json, orc, h5, hdf5')
        path, extension = filename.split('.')
        if '/' in path:
            path_list = filename.split('/')
            name = path_list[-1]
            print('path was given. name is {}'.format(name))
        else:
            name = filename
            print('only filename was given. name is {}'.format(name))
        if extension == 'csv' or extension == 'txt':
            data = pd.read_csv(name, **kwargs)
        if extension == 'ods' or extension == 'odt' or extension == 'xls' or extension == 'xlsx' or extension == 'xlsm' or extension == 'xlsb' or extension == 'odf':
            data = pd.read_excel(name, **kwargs)
        if extension == 'html':
            data = pd.read_html(name, **kwargs)
        if extension == 'fwf':
            data = pd.read_fwf(name, **kwargs)
        if extension == 'json':
            data = pdread_json(name, **kwargs)
        if extension == 'orc':
            data = pd.read_orc(name, **kwargs)
        if extension == 'h5' or extension == 'hdf5':
            assert '/' in path, 'pandas hdf reader requires filepath'
            data = pd.read_hdf(filename, **kwargs)
        return data
 
    def fix_elements(data): #technically newest_data
    """
    calculates abundance ratios for star/thing that we care about given a dataset
    keys have already been fixed using fix_key for indexing and matching to solar_abundance
    """
        g1 = data['Fe/H'] #g1, for given data value #1 -- number density of iron to hydrogen of star/thing we care about
        s1 = solar_abundance['Fe'] #s1, for solar data value #1 -- number density of iron to hydrogen of Sun
        t1 = s1*(10**g1)
        dic = {}
        dic['Fe'] = t1
        for key in solar_abundance.keys():
            if key in data.keys():
                if key == "Fe": # since we have already fixed the keys to match keys of SolAb, this should work
                    pass
                else:
                    t2 = (solar_abundance[key] / s1) * (10**data[key])
                    T = t1 * t2
                    dic[key] = T
        return dic
    def fix_key(data): 
        """
        amends element keys to ensure they match that of solar 
        abundances dictionary for indexing/comparing purposes
        Handy for data given in ratios 
        Returns new dictionary with updated keys and unchanged elements
        """
        """
        convert data to dictionary and ensure each entry is a float and not a list
        """
        new_data = data.to_dict(orient='list') 
        for i in new_data:
            new_data[i] = sum(new_data[i])
        """
        Make list of keys from data dictionary
        """ 
        keys = list(new_data.keys())
        string = []
        for i in keys:
            if len(i) == 1 or i[1] == '/':
                string.append(i[0])
            else:
                string.append(i[0] + i[1])
        """
        Make new dictionary with new keys and unchanged elements
        """
        newest_data = {}
        for key, i in zip(new_data.keys(), string):
            newest_data[i] = new_data[key]
        return newest_data
    
    def find_elements(filename, abundance, modify_solar = False, ratios = True, **kwargs):
        """
        fixes keys to be consistent with format of keys in solar_abundance

        If modify_solar = False:
        if ratios = False:
        -No values are changed. Returns a dictionary of data provided to be passed to ion_balance. 
        if ratios = True:
        -fix_elements is called. Returns a dictionary of corrected ratios to be passed to ion_balance
        If modify_solar = True:
        if ratios = False:
        -matches keys of data provided with keys of solar_abundance. If a match is found, replaces 
        value in solar_abundance with value in data provided. Returns solar_abundance dictionary with updata
        values to pass to ion_balance
        if ratios = True: 
        -fix_elements is called. Uses dictionary of corrected ratios to match keys of fixed data to keys 
        -of solar_abundance. If a match is found, replaces value in solar_abundance with value from 
        -fix_elements dictionary. Returns solar_abundance dictionary with updated values to pass to 
        -ion_balance

        ratios should only be False if data provided has already been corrected 
        """
        data = read_filetype(filename, **kwargs) 
        newest_data = fix_keys(data)
        if modify_solar == False:
            if ratios == False:
                adjusted_data = newest_data
            if ratios is not False:
                adjusted_data = fix_elements(newest_data)
            return adjusted_data
        if modify_solar is not False:
            if ratios == False:
                for key in abundance.keys():
                    if key in newest_data.keys():
                        abundance[key] = newest_data[key]
                        print('an element was changed to {}'.format(abundance[key]))
            if ratios is not False:
                adjusted_data = fix_elements(newest_data)
                for key in abundance.keys():
                    if key in adjusted_data.keys():
                        abundance[key] = adjusted_data[key]
                        print('an element was changed to {}'.format(abundance[key]))
            return abundance

    abundance = {
        'H' : 1.00e+00, 'He': 1.00e-01, 'Li': 2.04e-09,
        'Be': 2.63e-11, 'B' : 6.17e-10, 'C' : 2.45e-04,
        'N' : 8.51e-05, 'O' : 4.90e-04, 'F' : 3.02e-08,
        'Ne': 1.00e-04, 'Na': 2.14e-06, 'Mg': 3.47e-05,
        'Al': 2.95e-06, 'Si': 3.47e-05, 'P' : 3.20e-07,
        'S' : 1.84e-05, 'Cl': 1.91e-07, 'Ar': 2.51e-06,
        'K' : 1.32e-07, 'Ca': 2.29e-06, 'Sc': 1.48e-09,
        'Ti': 1.05e-07, 'V' : 1.00e-08, 'Cr': 4.68e-07,
        'Mn': 2.88e-07, 'Fe': 2.82e-05, 'Co': 8.32e-08,
        'Ni': 1.78e-06, 'Cu': 1.62e-08, 'Zn': 3.98e-08}
    
    if filename is not None:
        if modify_solar == False:
            if ratios == False:
                abundance = find_elements(filename, abundance, modify_solar=False, ratios = False, **kwargs)
            if ratios is not False:
                abundance = find_elements(filename, abundance, modify_solar=False, ratios = True, **kwargs)
        if modify_solar is not False:
            if ratios == False:
                abundance = find_elements(filename, abundance, modify_solar=True, ratios = False, **kwargs)
            if ratios is not False:
                abundance = find_elements(filename, abundance, modify_solar=True, ratios = True, **kwargs)
    return abundance

In [5]:
H = 'H'
h = 'h'
if H == h:
    print('yuh')
else:
    print('string indexing is case sensitive')

string indexing is case sensitive
